In [11]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import random
import os
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import warnings
warnings.filterwarnings('ignore')

In [12]:
df_names = ['train_features', 'train_targets_scored', 'train_targets_nonscored', 'test_features', 'sample_submission']

df = {}
for name in df_names:
    df[name] = pd.read_csv(f"../input/lish-moa/{name}.csv", index_col=0)
    print(f"{name}: {df[name].shape}")

train_features: (23814, 875)
train_targets_scored: (23814, 206)
train_targets_nonscored: (23814, 402)
test_features: (3982, 875)
sample_submission: (3982, 206)


In [13]:
df_full = pd.concat([df['train_features'], df['train_targets_scored'].add_prefix('scored_'), df['train_targets_nonscored'].add_prefix('nonscored_')], axis=1)
df_full['count_scored'] = df_full.filter(regex='^scored_').sum(axis=1)
df_full['count_nonscored'] = df_full.filter(regex='^nonscored_').sum(axis=1)
df_full['count_total'] = df_full['count_scored'] + df_full['count_nonscored']


In [14]:
total_target_cols = list(df_full.filter(regex='^(scored|nonscored)_').columns)
print("length of total targets, scored and non scored:", len(total_target_cols))
df_target_combine = df_full[["cp_type"]+total_target_cols].groupby(total_target_cols).size().reset_index(name='Combination_Count').sort_values('Combination_Count', ascending=False).reset_index(drop=True)

length of total targets, scored and non scored: 608


In [15]:
df_target_combine['Combination_Label'] = df_target_combine.index
df_target_combine = df_target_combine[['Combination_Label', 'Combination_Count'] + total_target_cols]
df_target_combine.head()

,Combination_Label,Combination_Count,scored_5-alpha_reductase_inhibitor,scored_11-beta-hsd1_inhibitor,scored_acat_inhibitor,scored_acetylcholine_receptor_agonist,scored_acetylcholine_receptor_antagonist,scored_acetylcholinesterase_inhibitor,scored_adenosine_receptor_agonist,scored_adenosine_receptor_antagonist,scored_adenylyl_cyclase_activator,scored_adrenergic_receptor_agonist,scored_adrenergic_receptor_antagonist,scored_akt_inhibitor,scored_aldehyde_dehydrogenase_inhibitor,scored_alk_inhibitor,scored_ampk_activator,scored_analgesic,scored_androgen_receptor_agonist,scored_androgen_receptor_antagonist,scored_anesthetic_-_local,scored_angiogenesis_inhibitor,scored_angiotensin_receptor_antagonist,scored_anti-inflammatory,scored_antiarrhythmic,scored_antibiotic,scored_anticonvulsant,scored_antifungal,scored_antihistamine,scored_antimalarial,scored_antioxidant,scored_antiprotozoal,scored_antiviral,scored_apoptosis_stimulant,scored_aromatase_inhibitor,scored_atm_kinase_inhibitor,scored_atp-sensitive_potassium_channel_antagonist,scored_atp_synthase_inhibitor,scored_atpase_inhibitor,scored_atr_kinase_inhibitor,scored_aurora_kinase_inhibitor,scored_autotaxin_inhibitor,scored_bacterial_30s_ribosomal_subunit_inhibitor,scored_bacterial_50s_ribosomal_subunit_inhibitor,scored_bacterial_antifolate,scored_bacterial_cell_wall_synthesis_inhibitor,scored_bacterial_dna_gyrase_inhibitor,scored_bacterial_dna_inhibitor,scored_bacterial_membrane_integrity_inhibitor,scored_bcl_inhibitor,scored_bcr-abl_inhibitor,scored_benzodiazepine_receptor_agonist,scored_beta_amyloid_inhibitor,scored_bromodomain_inhibitor,scored_btk_inhibitor,scored_calcineurin_inhibitor,scored_calcium_channel_blocker,scored_cannabinoid_receptor_agonist,scored_cannabinoid_receptor_antagonist,scored_carbonic_anhydrase_inhibitor,scored_casein_kinase_inhibitor,scored_caspase_activator,scored_catechol_o_methyltransferase_inhibitor,scored_cc_chemokine_receptor_antagonist,scored_cck_receptor_antagonist,scored_cdk_inhibitor,scored_chelating_agent,scored_chk_inhibitor,scored_chloride_channel_blocker,scored_cholesterol_inhibitor,scored_cholinergic_receptor_antagonist,scored_coagulation_factor_inhibitor,scored_corticosteroid_agonist,scored_cyclooxygenase_inhibitor,scored_cytochrome_p450_inhibitor,scored_dihydrofolate_reductase_inhibitor,scored_dipeptidyl_peptidase_inhibitor,scored_diuretic,scored_dna_alkylating_agent,scored_dna_inhibitor,scored_dopamine_receptor_agonist,scored_dopamine_receptor_antagonist,scored_egfr_inhibitor,scored_elastase_inhibitor,scored_erbb2_inhibitor,scored_estrogen_receptor_agonist,scored_estrogen_receptor_antagonist,scored_faah_inhibitor,scored_farnesyltransferase_inhibitor,scored_fatty_acid_receptor_agonist,scored_fgfr_inhibitor,scored_flt3_inhibitor,scored_focal_adhesion_kinase_inhibitor,scored_free_radical_scavenger,scored_fungal_squalene_epoxidase_inhibitor,scored_gaba_receptor_agonist,scored_gaba_receptor_antagonist,scored_gamma_secretase_inhibitor,scored_glucocorticoid_receptor_agonist,scored_glutamate_inhibitor,scored_glutamate_receptor_agonist,scored_glutamate_receptor_antagonist,scored_gonadotropin_receptor_agonist,scored_gsk_inhibitor,scored_hcv_inhibitor,scored_hdac_inhibitor,scored_histamine_receptor_agonist,scored_histamine_receptor_antagonist,scored_histone_lysine_demethylase_inhibitor,scored_histone_lysine_methyltransferase_inhibitor,scored_hiv_inhibitor,scored_hmgcr_inhibitor,scored_hsp_inhibitor,scored_igf-1_inhibitor,scored_ikk_inhibitor,scored_imidazoline_receptor_agonist,scored_immunosuppressant,scored_insulin_secretagogue,scored_insulin_sensitizer,scored_integrin_inhibitor,scored_jak_inhibitor,scored_kit_inhibitor,scored_laxative,scored_leukotriene_inhibitor,scored_leukotriene_receptor_antagonist,scored_lipase_inhibitor,scored_lipoxygenase_inhibitor,scored_lxr_agonist,scored_mdm_inhibitor,scored_mek_inhibitor,scored_membrane_integrity_inhibitor,scored_mineralocorticoid_receptor_antagonist,scored_monoacylglycerol_lipase_inhibitor,scored_monoamin

In [16]:
transition = df_target_combine[['Combination_Label']+total_target_cols].set_index('Combination_Label')
transition.head()

,scored_5-alpha_reductase_inhibitor,scored_11-beta-hsd1_inhibitor,scored_acat_inhibitor,scored_acetylcholine_receptor_agonist,scored_acetylcholine_receptor_antagonist,scored_acetylcholinesterase_inhibitor,scored_adenosine_receptor_agonist,scored_adenosine_receptor_antagonist,scored_adenylyl_cyclase_activator,scored_adrenergic_receptor_agonist,scored_adrenergic_receptor_antagonist,scored_akt_inhibitor,scored_aldehyde_dehydrogenase_inhibitor,scored_alk_inhibitor,scored_ampk_activator,scored_analgesic,scored_androgen_receptor_agonist,scored_androgen_receptor_antagonist,scored_anesthetic_-_local,scored_angiogenesis_inhibitor,scored_angiotensin_receptor_antagonist,scored_anti-inflammatory,scored_antiarrhythmic,scored_antibiotic,scored_anticonvulsant,scored_antifungal,scored_antihistamine,scored_antimalarial,scored_antioxidant,scored_antiprotozoal,scored_antiviral,scored_apoptosis_stimulant,scored_aromatase_inhibitor,scored_atm_kinase_inhibitor,scored_atp-sensitive_potassium_channel_antagonist,scored_atp_synthase_inhibitor,scored_atpase_inhibitor,scored_atr_kinase_inhibitor,scored_aurora_kinase_inhibitor,scored_autotaxin_inhibitor,scored_bacterial_30s_ribosomal_subunit_inhibitor,scored_bacterial_50s_ribosomal_subunit_inhibitor,scored_bacterial_antifolate,scored_bacterial_cell_wall_synthesis_inhibitor,scored_bacterial_dna_gyrase_inhibitor,scored_bacterial_dna_inhibitor,scored_bacterial_membrane_integrity_inhibitor,scored_bcl_inhibitor,scored_bcr-abl_inhibitor,scored_benzodiazepine_receptor_agonist,scored_beta_amyloid_inhibitor,scored_bromodomain_inhibitor,scored_btk_inhibitor,scored_calcineurin_inhibitor,scored_calcium_channel_blocker,scored_cannabinoid_receptor_agonist,scored_cannabinoid_receptor_antagonist,scored_carbonic_anhydrase_inhibitor,scored_casein_kinase_inhibitor,scored_caspase_activator,scored_catechol_o_methyltransferase_inhibitor,scored_cc_chemokine_receptor_antagonist,scored_cck_receptor_antagonist,scored_cdk_inhibitor,scored_chelating_agent,scored_chk_inhibitor,scored_chloride_channel_blocker,scored_cholesterol_inhibitor,scored_cholinergic_receptor_antagonist,scored_coagulation_factor_inhibitor,scored_corticosteroid_agonist,scored_cyclooxygenase_inhibitor,scored_cytochrome_p450_inhibitor,scored_dihydrofolate_reductase_inhibitor,scored_dipeptidyl_peptidase_inhibitor,scored_diuretic,scored_dna_alkylating_agent,scored_dna_inhibitor,scored_dopamine_receptor_agonist,scored_dopamine_receptor_antagonist,scored_egfr_inhibitor,scored_elastase_inhibitor,scored_erbb2_inhibitor,scored_estrogen_receptor_agonist,scored_estrogen_receptor_antagonist,scored_faah_inhibitor,scored_farnesyltransferase_inhibitor,scored_fatty_acid_receptor_agonist,scored_fgfr_inhibitor,scored_flt3_inhibitor,scored_focal_adhesion_kinase_inhibitor,scored_free_radical_scavenger,scored_fungal_squalene_epoxidase_inhibitor,scored_gaba_receptor_agonist,scored_gaba_receptor_antagonist,scored_gamma_secretase_inhibitor,scored_glucocorticoid_receptor_agonist,scored_glutamate_inhibitor,scored_glutamate_receptor_agonist,scored_glutamate_receptor_antagonist,scored_gonadotropin_receptor_agonist,scored_gsk_inhibitor,scored_hcv_inhibitor,scored_hdac_inhibitor,scored_histamine_receptor_agonist,scored_histamine_receptor_antagonist,scored_histone_lysine_demethylase_inhibitor,scored_histone_lysine_methyltransferase_inhibitor,scored_hiv_inhibitor,scored_hmgcr_inhibitor,scored_hsp_inhibitor,scored_igf-1_inhibitor,scored_ikk_inhibitor,scored_imidazoline_receptor_agonist,scored_immunosuppressant,scored_insulin_secretagogue,scored_insulin_sensitizer,scored_integrin_inhibitor,scored_jak_inhibitor,scored_kit_inhibitor,scored_laxative,scored_leukotriene_inhibitor,scored_leukotriene_receptor_antagonist,scored_lipase_inhibitor,scored_lipoxygenase_inhibitor,scored_lxr_agonist,scored_mdm_inhibitor,scored_mek_inhibitor,scored_membrane_integrity_inhibitor,scored_mineralocorticoid_receptor_antagonist,scored_monoacylglycerol_lipase_inhibitor,scored_monoamine_oxidase_inhibitor,scored_monopolar

In [17]:
df_full_combine = pd.merge(df_full, df_target_combine, on=total_target_cols, how='left')
df_full_combine.head()

,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,g-7,g-8,g-9,g-10,g-11,g-12,g-13,g-14,g-15,g-16,g-17,g-18,g-19,g-20,g-21,g-22,g-23,g-24,g-25,g-26,g-27,g-28,g-29,g-30,g-31,g-32,g-33,g-34,g-35,g-36,g-37,g-38,g-39,g-40,g-41,g-42,g-43,g-44,g-45,g-46,g-47,g-48,g-49,g-50,g-51,g-52,g-53,g-54,g-55,g-56,g-57,g-58,g-59,g-60,g-61,g-62,g-63,g-64,g-65,g-66,g-67,g-68,g-69,g-70,g-71,g-72,g-73,g-74,g-75,g-76,g-77,g-78,g-79,g-80,g-81,g-82,g-83,g-84,g-85,g-86,g-87,g-88,g-89,g-90,g-91,g-92,g-93,g-94,g-95,g-96,g-97,g-98,g-99,g-100,g-101,g-102,g-103,g-104,g-105,g-106,g-107,g-108,g-109,g-110,g-111,g-112,g-113,g-114,g-115,g-116,g-117,g-118,g-119,g-120,g-121,g-122,g-123,g-124,g-125,g-126,g-127,g-128,g-129,g-130,g-131,g-132,g-133,g-134,g-135,g-136,g-137,g-138,g-139,g-140,g-141,g-142,g-143,g-144,g-145,g-146,g-147,g-148,g-149,g-150,g-151,g-152,g-153,g-154,g-155,g-156,g-157,g-158,g-159,g-160,g-161,g-162,g-163,g-164,g-165,g-166,g-167,g-168,g-169,g-170,g-171,g-172,g-173,g-174,g-175,g-176,g-177,g-178,g-179,g-180,g-181,g-182,g-183,g-184,g-185,g-186,g-187,g-188,g-189,g-190,g-191,g-192,g-193,g-194,g-195,g-196,g-197,g-198,g-199,g-200,g-201,g-202,g-203,g-204,g-205,g-206,g-207,g-208,g-209,g-210,g-211,g-212,g-213,g-214,g-215,g-216,g-217,g-218,g-219,g-220,g-221,g-222,g-223,g-224,g-225,g-226,g-227,g-228,g-229,g-230,g-231,g-232,g-233,g-234,g-235,g-236,g-237,g-238,g-239,g-240,g-241,g-242,g-243,g-244,g-245,g-246,g-247,g-248,g-249,g-250,g-251,g-252,g-253,g-254,g-255,g-256,g-257,g-258,g-259,g-260,g-261,g-262,g-263,g-264,g-265,g-266,g-267,g-268,g-269,g-270,g-271,g-272,g-273,g-274,g-275,g-276,g-277,g-278,g-279,g-280,g-281,g-282,g-283,g-284,g-285,g-286,g-287,g-288,g-289,g-290,g-291,g-292,g-293,g-294,g-295,g-296,g-297,g-298,g-299,g-300,g-301,g-302,g-303,g-304,g-305,g-306,g-307,g-308,g-309,g-310,g-311,g-312,g-313,g-314,g-315,g-316,g-317,g-318,g-319,g-320,g-321,g-322,g-323,g-324,g-325,g-326,g-327,g-328,g-329,g-330,g-331,g-332,g-333,g-334,g-335,g-336,g-337,g-338,g-339,g-340,g-341,g-342,g-343,g-344,g-345,g-346,g-347,g-348,g-349,g-350,g-351,g-352,g-353,g-354,g-355,g-356,g-357,g-358,g-359,g-360,g-361,g-362,g-363,g-364,g-365,g-366,g-367,g-368,g-369,g-370,g-371,g-372,g-373,g-374,g-375,g-376,g-377,g-378,g-379,g-380,g-381,g-382,g-383,g-384,g-385,g-386,g-387,g-388,g-389,g-390,g-391,g-392,g-393,g-394,g-395,g-396,g-397,g-398,g-399,g-400,g-401,g-402,g-403,g-404,g-405,g-406,g-407,g-408,g-409,g-410,g-411,g-412,g-413,g-414,g-415,g-416,g-417,g-418,g-419,g-420,g-421,g-422,g-423,g-424,g-425,g-426,g-427,g-428,g-429,g-430,g-431,g-432,g-433,g-434,g-435,g-436,g-437,g-438,g-439,g-440,g-441,g-442,g-443,g-444,g-445,g-446,g-447,g-448,g-449,g-450,g-451,g-452,g-453,g-454,g-455,g-456,g-457,g-458,g-459,g-460,g-461,g-462,g-463,g-464,g-465,g-466,g-467,g-468,g-469,g-470,g-471,g-472,g-473,g-474,g-475,g-476,g-477,g-478,g-479,g-480,g-481,g-482,g-483,g-484,g-485,g-486,g-487,g-488,g-489,g-490,g-491,g-492,g-493,g-494,g-495,g-496,g-497,g-498,g-499,g-500,g-501,g-502,g-503,g-504,g-505,g-506,g-507,g-508,g-509,g-510,g-511,g-512,g-513,g-514,g-515,g-516,g-517,g-518,g-519,g-520,g-521,g-522,g-523,g-524,g-525,g-526,g-527,g-528,g-529,g-530,g-531,g-532,g-533,g-534,g-535,g-536,g-537,g-538,g-539,g-540,g-541,g-542,g-543,g-544,g-545,g-546,g-547,g-548,g-549,g-550,g-551,g-552,g-553,g-554,g-555,g-556,g-557,g-558,g-559,g-560,g-561,g-562,g-563,g-564,g-565,g-566,g-567,g-568,g-569,g-570,g-571,g-572,g-573,g-574,g-575,g-576,g-577,g-578,g-579,g-580,g-581,g-582,g-583,g-584,g-585,g-586,g-587,g-588,g-589,g-590,g-591,g-592,g-593,g-594,g-595,g-596,g-597,g-598,g-599,g-600,g-601,g-602,g-603,g-604,g-605,g-606,g-607,g-608,g-609,g-610,g-611,g-612,g-613,g-614,g-615,g-616,g-617,g-618,g-619,g-620,g-621,g-622,g-623,g-624,g-625,g-626,g-627,g-628,g-629,g-630,g-631,g-632,g-633,g-634,g-635,g-636,g-637,g-638,g-639,g-640,g-641,g-642,g-643,g-644,g-645,g-646,g-647,g-648,g-649,g-650,g-651,g-652,g-653,g-654,g-655,g-656,g-657,g-658,g-659,g-660,g-661,g-662,g-663,g-664,g-665,g-666,g-667,g-668,g-669,g-670,g-671,g-672,g-673,g-674,g-675,g-676,g-677,g-678,g-679,g-680

In [18]:
df_full_trt = df_full_combine[df_full_combine.cp_type !='ctl_vehicle']
df_train = df_full_trt.drop(columns= total_target_cols+['count_scored',
       'count_nonscored', 'count_total', 
       'Combination_Count'] ).reset_index(drop = True).drop('cp_type', axis = 1)

#df_test = df['test_features'][df['test_features']['cp_type']!='ctl_vehicle'].reset_index(drop = True).drop('cp_type', axis = 1)
df_test = df['test_features'].drop('cp_type', axis = 1)

In [19]:
combination_label_target = df_full_trt.Combination_Label.reset_index(drop = True)
df_target_OHE = pd.get_dummies(combination_label_target, prefix='combo')

In [20]:
df_train.head()

,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,g-7,g-8,g-9,g-10,g-11,g-12,g-13,g-14,g-15,g-16,g-17,g-18,g-19,g-20,g-21,g-22,g-23,g-24,g-25,g-26,g-27,g-28,g-29,g-30,g-31,g-32,g-33,g-34,g-35,g-36,g-37,g-38,g-39,g-40,g-41,g-42,g-43,g-44,g-45,g-46,g-47,g-48,g-49,g-50,g-51,g-52,g-53,g-54,g-55,g-56,g-57,g-58,g-59,g-60,g-61,g-62,g-63,g-64,g-65,g-66,g-67,g-68,g-69,g-70,g-71,g-72,g-73,g-74,g-75,g-76,g-77,g-78,g-79,g-80,g-81,g-82,g-83,g-84,g-85,g-86,g-87,g-88,g-89,g-90,g-91,g-92,g-93,g-94,g-95,g-96,g-97,g-98,g-99,g-100,g-101,g-102,g-103,g-104,g-105,g-106,g-107,g-108,g-109,g-110,g-111,g-112,g-113,g-114,g-115,g-116,g-117,g-118,g-119,g-120,g-121,g-122,g-123,g-124,g-125,g-126,g-127,g-128,g-129,g-130,g-131,g-132,g-133,g-134,g-135,g-136,g-137,g-138,g-139,g-140,g-141,g-142,g-143,g-144,g-145,g-146,g-147,g-148,g-149,g-150,g-151,g-152,g-153,g-154,g-155,g-156,g-157,g-158,g-159,g-160,g-161,g-162,g-163,g-164,g-165,g-166,g-167,g-168,g-169,g-170,g-171,g-172,g-173,g-174,g-175,g-176,g-177,g-178,g-179,g-180,g-181,g-182,g-183,g-184,g-185,g-186,g-187,g-188,g-189,g-190,g-191,g-192,g-193,g-194,g-195,g-196,g-197,g-198,g-199,g-200,g-201,g-202,g-203,g-204,g-205,g-206,g-207,g-208,g-209,g-210,g-211,g-212,g-213,g-214,g-215,g-216,g-217,g-218,g-219,g-220,g-221,g-222,g-223,g-224,g-225,g-226,g-227,g-228,g-229,g-230,g-231,g-232,g-233,g-234,g-235,g-236,g-237,g-238,g-239,g-240,g-241,g-242,g-243,g-244,g-245,g-246,g-247,g-248,g-249,g-250,g-251,g-252,g-253,g-254,g-255,g-256,g-257,g-258,g-259,g-260,g-261,g-262,g-263,g-264,g-265,g-266,g-267,g-268,g-269,g-270,g-271,g-272,g-273,g-274,g-275,g-276,g-277,g-278,g-279,g-280,g-281,g-282,g-283,g-284,g-285,g-286,g-287,g-288,g-289,g-290,g-291,g-292,g-293,g-294,g-295,g-296,g-297,g-298,g-299,g-300,g-301,g-302,g-303,g-304,g-305,g-306,g-307,g-308,g-309,g-310,g-311,g-312,g-313,g-314,g-315,g-316,g-317,g-318,g-319,g-320,g-321,g-322,g-323,g-324,g-325,g-326,g-327,g-328,g-329,g-330,g-331,g-332,g-333,g-334,g-335,g-336,g-337,g-338,g-339,g-340,g-341,g-342,g-343,g-344,g-345,g-346,g-347,g-348,g-349,g-350,g-351,g-352,g-353,g-354,g-355,g-356,g-357,g-358,g-359,g-360,g-361,g-362,g-363,g-364,g-365,g-366,g-367,g-368,g-369,g-370,g-371,g-372,g-373,g-374,g-375,g-376,g-377,g-378,g-379,g-380,g-381,g-382,g-383,g-384,g-385,g-386,g-387,g-388,g-389,g-390,g-391,g-392,g-393,g-394,g-395,g-396,g-397,g-398,g-399,g-400,g-401,g-402,g-403,g-404,g-405,g-406,g-407,g-408,g-409,g-410,g-411,g-412,g-413,g-414,g-415,g-416,g-417,g-418,g-419,g-420,g-421,g-422,g-423,g-424,g-425,g-426,g-427,g-428,g-429,g-430,g-431,g-432,g-433,g-434,g-435,g-436,g-437,g-438,g-439,g-440,g-441,g-442,g-443,g-444,g-445,g-446,g-447,g-448,g-449,g-450,g-451,g-452,g-453,g-454,g-455,g-456,g-457,g-458,g-459,g-460,g-461,g-462,g-463,g-464,g-465,g-466,g-467,g-468,g-469,g-470,g-471,g-472,g-473,g-474,g-475,g-476,g-477,g-478,g-479,g-480,g-481,g-482,g-483,g-484,g-485,g-486,g-487,g-488,g-489,g-490,g-491,g-492,g-493,g-494,g-495,g-496,g-497,g-498,g-499,g-500,g-501,g-502,g-503,g-504,g-505,g-506,g-507,g-508,g-509,g-510,g-511,g-512,g-513,g-514,g-515,g-516,g-517,g-518,g-519,g-520,g-521,g-522,g-523,g-524,g-525,g-526,g-527,g-528,g-529,g-530,g-531,g-532,g-533,g-534,g-535,g-536,g-537,g-538,g-539,g-540,g-541,g-542,g-543,g-544,g-545,g-546,g-547,g-548,g-549,g-550,g-551,g-552,g-553,g-554,g-555,g-556,g-557,g-558,g-559,g-560,g-561,g-562,g-563,g-564,g-565,g-566,g-567,g-568,g-569,g-570,g-571,g-572,g-573,g-574,g-575,g-576,g-577,g-578,g-579,g-580,g-581,g-582,g-583,g-584,g-585,g-586,g-587,g-588,g-589,g-590,g-591,g-592,g-593,g-594,g-595,g-596,g-597,g-598,g-599,g-600,g-601,g-602,g-603,g-604,g-605,g-606,g-607,g-608,g-609,g-610,g-611,g-612,g-613,g-614,g-615,g-616,g-617,g-618,g-619,g-620,g-621,g-622,g-623,g-624,g-625,g-626,g-627,g-628,g-629,g-630,g-631,g-632,g-633,g-634,g-635,g-636,g-637,g-638,g-639,g-640,g-641,g-642,g-643,g-644,g-645,g-646,g-647,g-648,g-649,g-650,g-651,g-652,g-653,g-654,g-655,g-656,g-657,g-658,g-659,g-660,g-661,g-662,g-663,g-664,g-665,g-666,g-667,g-668,g-669,g-670,g-671,g-672,g-673,g-674,g-675,g-676,g-677,g-678,g-679,g-680,g-681,g

In [21]:
df_target_OHE.head()

,combo_0,combo_1,combo_2,combo_3,combo_4,combo_5,combo_6,combo_7,combo_8,combo_9,combo_10,combo_11,combo_12,combo_13,combo_14,combo_15,combo_16,combo_17,combo_18,combo_19,combo_20,combo_21,combo_22,combo_23,combo_24,combo_25,combo_26,combo_27,combo_28,combo_29,combo_30,combo_31,combo_32,combo_33,combo_34,combo_35,combo_36,combo_37,combo_38,combo_39,combo_40,combo_41,combo_42,combo_43,combo_44,combo_45,combo_46,combo_47,combo_48,combo_49,combo_50,combo_51,combo_52,combo_53,combo_54,combo_55,combo_56,combo_57,combo_58,combo_59,combo_60,combo_61,combo_62,combo_63,combo_64,combo_65,combo_66,combo_67,combo_68,combo_69,combo_70,combo_71,combo_72,combo_73,combo_74,combo_75,combo_76,combo_77,combo_78,combo_79,combo_80,combo_81,combo_82,combo_83,combo_84,combo_85,combo_86,combo_87,combo_88,combo_89,combo_90,combo_91,combo_92,combo_93,combo_94,combo_95,combo_96,combo_97,combo_98,combo_99,combo_100,combo_101,combo_102,combo_103,combo_104,combo_105,combo_106,combo_107,combo_108,combo_109,combo_110,combo_111,combo_112,combo_113,combo_114,combo_115,combo_116,combo_117,combo_118,combo_119,combo_120,combo_121,combo_122,combo_123,combo_124,combo_125,combo_126,combo_127,combo_128,combo_129,combo_130,combo_131,combo_132,combo_133,combo_134,combo_135,combo_136,combo_137,combo_138,combo_139,combo_140,combo_141,combo_142,combo_143,combo_144,combo_145,combo_146,combo_147,combo_148,combo_149,combo_150,combo_151,combo_152,combo_153,combo_154,combo_155,combo_156,combo_157,combo_158,combo_159,combo_160,combo_161,combo_162,combo_163,combo_164,combo_165,combo_166,combo_167,combo_168,combo_169,combo_170,combo_171,combo_172,combo_173,combo_174,combo_175,combo_176,combo_177,combo_178,combo_179,combo_180,combo_181,combo_182,combo_183,combo_184,combo_185,combo_186,combo_187,combo_188,combo_189,combo_190,combo_191,combo_192,combo_193,combo_194,combo_195,combo_196,combo_197,combo_198,combo_199,combo_200,combo_201,combo_202,combo_203,combo_204,combo_205,combo_206,combo_207,combo_208,combo_209,combo_210,combo_211,combo_212,combo_213,combo_214,combo_215,combo_216,combo_217,combo_218,combo_219,combo_220,combo_221,combo_222,combo_223,combo_224,combo_225,combo_226,combo_227,combo_228,combo_229,combo_230,combo_231,combo_232,combo_233,combo_234,combo_235,combo_236,combo_237,combo_238,combo_239,combo_240,combo_241,combo_242,combo_243,combo_244,combo_245,combo_246,combo_247,combo_248,combo_249,combo_250,combo_251,combo_252,combo_253,combo_254,combo_255,combo_256,combo_257,combo_258,combo_259,combo_260,combo_261,combo_262,combo_263,combo_264,combo_265,combo_266,combo_267,combo_268,combo_269,combo_270,combo_271,combo_272,combo_273,combo_274,combo_275,combo_276,combo_277,combo_278,combo_279,combo_280,combo_281,combo_282,combo_283,combo_284,combo_285,combo_286,combo_287,combo_288,combo_289,combo_290,combo_291,combo_292,combo_293,combo_294,combo_295,combo_296,combo_297,combo_298,combo_299,combo_300,combo_301,combo_302,combo_303,combo_304,combo_305,combo_306,combo_307,combo_308,combo_309,combo_310,combo_311,combo_312,combo_313,combo_314,combo_315,combo_316,combo_317,combo_318,combo_319,combo_320,combo_321,combo_322,combo_323,combo_324,combo_325,combo_326,combo_327,combo_328,combo_329,combo_330,combo_331,combo_332,combo_333,combo_334,combo_335,combo_336,combo_337,combo_338,combo_339,combo_340,combo_341,combo_342,combo_343,combo_344,combo_345,combo_346,combo_347,combo_348,combo_349,combo_350,combo_351,combo_352,combo_353,combo_354,combo_355,combo_356,combo_357,combo_358,combo_359,combo_360,combo_361,combo_362,combo_363,combo_364,combo_365,combo_366,combo_367,combo_368,combo_369,combo_370,combo_371,combo_372,combo_373,combo_374,combo_375,combo_376,combo_377,combo_378,combo_379,combo_380,combo_381,combo_382,combo_383,combo_384,combo_385,combo_386,combo_387,combo_388,combo_389,combo_390,combo_391,combo_392,combo_393,combo_394,combo_395,combo_396,combo_397,combo_398,combo_399,combo_400,combo_401,combo_402,combo_403,combo_404,combo_405,combo_406,combo_407,combo_408,combo_409,combo_410

## CV fold

In [22]:
#from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, random_state=2020, shuffle= True)

In [23]:
folds = df_train.copy()

for f, (train_index, val_index)  in enumerate(skf.split(X= df_train, y=combination_label_target)):
    folds.loc[val_index, 'kfold'] = int(f)
    
folds['kfold'] =folds['kfold'].astype(int)
folds.head()

,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,g-7,g-8,g-9,g-10,g-11,g-12,g-13,g-14,g-15,g-16,g-17,g-18,g-19,g-20,g-21,g-22,g-23,g-24,g-25,g-26,g-27,g-28,g-29,g-30,g-31,g-32,g-33,g-34,g-35,g-36,g-37,g-38,g-39,g-40,g-41,g-42,g-43,g-44,g-45,g-46,g-47,g-48,g-49,g-50,g-51,g-52,g-53,g-54,g-55,g-56,g-57,g-58,g-59,g-60,g-61,g-62,g-63,g-64,g-65,g-66,g-67,g-68,g-69,g-70,g-71,g-72,g-73,g-74,g-75,g-76,g-77,g-78,g-79,g-80,g-81,g-82,g-83,g-84,g-85,g-86,g-87,g-88,g-89,g-90,g-91,g-92,g-93,g-94,g-95,g-96,g-97,g-98,g-99,g-100,g-101,g-102,g-103,g-104,g-105,g-106,g-107,g-108,g-109,g-110,g-111,g-112,g-113,g-114,g-115,g-116,g-117,g-118,g-119,g-120,g-121,g-122,g-123,g-124,g-125,g-126,g-127,g-128,g-129,g-130,g-131,g-132,g-133,g-134,g-135,g-136,g-137,g-138,g-139,g-140,g-141,g-142,g-143,g-144,g-145,g-146,g-147,g-148,g-149,g-150,g-151,g-152,g-153,g-154,g-155,g-156,g-157,g-158,g-159,g-160,g-161,g-162,g-163,g-164,g-165,g-166,g-167,g-168,g-169,g-170,g-171,g-172,g-173,g-174,g-175,g-176,g-177,g-178,g-179,g-180,g-181,g-182,g-183,g-184,g-185,g-186,g-187,g-188,g-189,g-190,g-191,g-192,g-193,g-194,g-195,g-196,g-197,g-198,g-199,g-200,g-201,g-202,g-203,g-204,g-205,g-206,g-207,g-208,g-209,g-210,g-211,g-212,g-213,g-214,g-215,g-216,g-217,g-218,g-219,g-220,g-221,g-222,g-223,g-224,g-225,g-226,g-227,g-228,g-229,g-230,g-231,g-232,g-233,g-234,g-235,g-236,g-237,g-238,g-239,g-240,g-241,g-242,g-243,g-244,g-245,g-246,g-247,g-248,g-249,g-250,g-251,g-252,g-253,g-254,g-255,g-256,g-257,g-258,g-259,g-260,g-261,g-262,g-263,g-264,g-265,g-266,g-267,g-268,g-269,g-270,g-271,g-272,g-273,g-274,g-275,g-276,g-277,g-278,g-279,g-280,g-281,g-282,g-283,g-284,g-285,g-286,g-287,g-288,g-289,g-290,g-291,g-292,g-293,g-294,g-295,g-296,g-297,g-298,g-299,g-300,g-301,g-302,g-303,g-304,g-305,g-306,g-307,g-308,g-309,g-310,g-311,g-312,g-313,g-314,g-315,g-316,g-317,g-318,g-319,g-320,g-321,g-322,g-323,g-324,g-325,g-326,g-327,g-328,g-329,g-330,g-331,g-332,g-333,g-334,g-335,g-336,g-337,g-338,g-339,g-340,g-341,g-342,g-343,g-344,g-345,g-346,g-347,g-348,g-349,g-350,g-351,g-352,g-353,g-354,g-355,g-356,g-357,g-358,g-359,g-360,g-361,g-362,g-363,g-364,g-365,g-366,g-367,g-368,g-369,g-370,g-371,g-372,g-373,g-374,g-375,g-376,g-377,g-378,g-379,g-380,g-381,g-382,g-383,g-384,g-385,g-386,g-387,g-388,g-389,g-390,g-391,g-392,g-393,g-394,g-395,g-396,g-397,g-398,g-399,g-400,g-401,g-402,g-403,g-404,g-405,g-406,g-407,g-408,g-409,g-410,g-411,g-412,g-413,g-414,g-415,g-416,g-417,g-418,g-419,g-420,g-421,g-422,g-423,g-424,g-425,g-426,g-427,g-428,g-429,g-430,g-431,g-432,g-433,g-434,g-435,g-436,g-437,g-438,g-439,g-440,g-441,g-442,g-443,g-444,g-445,g-446,g-447,g-448,g-449,g-450,g-451,g-452,g-453,g-454,g-455,g-456,g-457,g-458,g-459,g-460,g-461,g-462,g-463,g-464,g-465,g-466,g-467,g-468,g-469,g-470,g-471,g-472,g-473,g-474,g-475,g-476,g-477,g-478,g-479,g-480,g-481,g-482,g-483,g-484,g-485,g-486,g-487,g-488,g-489,g-490,g-491,g-492,g-493,g-494,g-495,g-496,g-497,g-498,g-499,g-500,g-501,g-502,g-503,g-504,g-505,g-506,g-507,g-508,g-509,g-510,g-511,g-512,g-513,g-514,g-515,g-516,g-517,g-518,g-519,g-520,g-521,g-522,g-523,g-524,g-525,g-526,g-527,g-528,g-529,g-530,g-531,g-532,g-533,g-534,g-535,g-536,g-537,g-538,g-539,g-540,g-541,g-542,g-543,g-544,g-545,g-546,g-547,g-548,g-549,g-550,g-551,g-552,g-553,g-554,g-555,g-556,g-557,g-558,g-559,g-560,g-561,g-562,g-563,g-564,g-565,g-566,g-567,g-568,g-569,g-570,g-571,g-572,g-573,g-574,g-575,g-576,g-577,g-578,g-579,g-580,g-581,g-582,g-583,g-584,g-585,g-586,g-587,g-588,g-589,g-590,g-591,g-592,g-593,g-594,g-595,g-596,g-597,g-598,g-599,g-600,g-601,g-602,g-603,g-604,g-605,g-606,g-607,g-608,g-609,g-610,g-611,g-612,g-613,g-614,g-615,g-616,g-617,g-618,g-619,g-620,g-621,g-622,g-623,g-624,g-625,g-626,g-627,g-628,g-629,g-630,g-631,g-632,g-633,g-634,g-635,g-636,g-637,g-638,g-639,g-640,g-641,g-642,g-643,g-644,g-645,g-646,g-647,g-648,g-649,g-650,g-651,g-652,g-653,g-654,g-655,g-656,g-657,g-658,g-659,g-660,g-661,g-662,g-663,g-664,g-665,g-666,g-667,g-668,g-669,g-670,g-671,g-672,g-673,g-674,g-675,g-676,g-677,g-678,g-679,g-680,g-681,g

## DataSet Class

In [24]:
class MoADataset:
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
    #the __len__ and __getitem__ are for torch.utils.data.DataLoader to load batches into neural networks.    
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float),
            'y' : torch.tensor(self.targets[idx, :], dtype=torch.float)            
        }
        return dct
    
class TestDataset:
    def __init__(self, features):
        self.features = features
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float)
        }
        return dct
    

In [25]:
def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device):
    # this is to tell the model that it is in train mode, thus use batch normalization and dropout.
    model.train()
    final_loss = 0
    
    for data in dataloader:
        optimizer.zero_grad()
        inputs, targets = data['x'].to(device), data['y'].to(device)
        
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        final_loss += loss.item()
        
    final_loss /= len(dataloader)
    
    return final_loss


def valid_fn(model, loss_fn, dataloader, device):
    #model.eval() is to disable batch normalization and dropout.
    model.eval()
    final_loss = 0
    valid_preds = []
    
    for data in dataloader:
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        
        final_loss += loss.item()
        valid_preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    final_loss /= len(dataloader)
    valid_preds = np.concatenate(valid_preds)
    
    return final_loss, valid_preds

def inference_fn(model, dataloader, device):
    model.eval()
    preds = []
    
    for data in dataloader:
        inputs = data['x'].to(device)
        
        #this saves memory and accelerate the running time as we don't need to keep memory for the gradients.
        with torch.no_grad():
            outputs = model(inputs)
        
        preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    preds = np.concatenate(preds)
    
    return preds
   
    

## Model

In [34]:
class Model(nn.Module):
    def __init__(self, num_features, num_targets, hidden_size):
        super(Model, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dropout1 = nn.Dropout(0.2)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size))
        
        self.batch_norm2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(0.3)
        self.dense2 = nn.utils.weight_norm(nn.Linear(hidden_size, hidden_size))
        
        self.batch_norm3 = nn.BatchNorm1d(hidden_size)
        self.dropout3 = nn.Dropout(0.2)
        self.dense3 = nn.utils.weight_norm(nn.Linear(hidden_size, num_targets))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = F.relu(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.relu(self.dense2(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.dense3(x)
        
        return x

In [27]:
def process_data(data):
    
    data = pd.get_dummies(data, columns=['cp_time','cp_dose'])
    return data

In [28]:
feature_cols = [c for c in process_data(folds).columns if c!='Combination_Label']
feature_cols = [c for c in feature_cols if c not in ['kfold','sig_id']]
target_cols = [c for c in df_target_OHE.columns]
print(len(feature_cols), len(target_cols))

877 696


In [29]:
def seed_everything(seed=2020):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=2020)

In [30]:
# HyperParameters

DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 25
BATCH_SIZE = 1024
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-5 #this is important for NN.
NFOLDS = 5
EARLY_STOPPING_STEPS = 20
EARLY_STOP = False

num_features=len(feature_cols)
num_targets=df_target_OHE.shape[1]
hidden_size=1024

## Single_fold_training

In [31]:
#need to modify the dataframe names
def run_training(fold, seed):
    
    seed_everything(seed)
    
    train = process_data(folds)
    test_ = process_data(df_test)
    
    trn_idx = train[train['kfold'] != fold].index
    val_idx = train[train['kfold'] == fold].index
    
    train_df = train[train['kfold'] != fold].reset_index(drop=True)
    valid_df = train[train['kfold'] == fold].reset_index(drop=True)
    
    train_target = df_target_OHE.iloc[trn_idx,:]
    val_target = df_target_OHE.iloc[val_idx,:]
    
    #x_train, y_train  = train_df[feature_cols].values, train_df[target_cols].values
    #x_valid, y_valid =  valid_df[feature_cols].values, valid_df[target_cols].values

    x_train, y_train = train_df[feature_cols].values, train_target.values
    x_valid, y_valid =  valid_df[feature_cols].values,val_target.values
    
    train_dataset = MoADataset(x_train, y_train)
    valid_dataset = MoADataset(x_valid, y_valid)
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets=num_targets,
        hidden_size=hidden_size,
    )
    
    model.to(DEVICE)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    #optimizer = torch.optim.RMsprop(model.parameters(), lr=LEARNING_RATE, alpha=0.99, eps=1e-08, weight_decay=0, momentum=0, centered=False)
    
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
                                              max_lr=1e-2, epochs=EPOCHS, steps_per_epoch=len(trainloader))
    
    loss_fn = nn.BCEWithLogitsLoss()
    
    early_stopping_steps = EARLY_STOPPING_STEPS
    early_step = 0
    
    oof = np.zeros((len(df_train), df_target_OHE.shape[1]))
    best_loss = np.inf
    
    for epoch in range(EPOCHS):
        
        train_loss = train_fn(model, optimizer,scheduler, loss_fn, trainloader, DEVICE)
        print(f"FOLD: {fold}, EPOCH: {epoch}, train_loss: {train_loss}")
        valid_loss, valid_preds = valid_fn(model, loss_fn, validloader, DEVICE)
        print(f"FOLD: {fold}, EPOCH: {epoch}, valid_loss: {valid_loss}")
        
        if valid_loss < best_loss:
            
            best_loss = valid_loss
            oof[val_idx] = valid_preds
            torch.save(model.state_dict(), f"FOLD{fold}_.pth")
        
        elif(EARLY_STOP == True):
            
            early_step += 1
            if (early_step >= early_stopping_steps):
                break
            
    """"
    as a first step we train the combination targets and
    just put the combination back using transition matrix of size (696* 608)
    
    Second step would be to use transition targets
    """
    
    #--------------------- PREDICTION---------------------
    x_test = test_[feature_cols].values
    testdataset = TestDataset(x_test)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets=num_targets,
        hidden_size=hidden_size,
    )
    
    model.load_state_dict(torch.load(f"FOLD{fold}_.pth"))
    model.to(DEVICE)
    
    #predictions = np.zeros((len(test_), target.iloc[:, 1:].shape[1]))
    predictions = np.zeros((len(test_), df_target_OHE.shape[1]))
    predictions = inference_fn(model, testloader, DEVICE)
    
    return oof, predictions
   

In [32]:
def run_k_fold(NFOLDS, seed):
    oof = np.zeros((len(df_train), df_target_OHE.shape[1]))
    predictions = np.zeros((len(df_test), df_target_OHE.shape[1] ))
    
    for fold in range(NFOLDS):
        oof_, pred_ = run_training(fold, seed)
        
        predictions += pred_ / NFOLDS
        oof += oof_
        
    return oof, predictions

In [36]:
SEED = [42,2020,2,27]
#SEED = [2]
oof = np.zeros((len(df_train),df_target_OHE.shape[1] ))
predictions = np.zeros((len(df_test), df_target_OHE.shape[1]))

for seed in SEED:
    
    oof_, predictions_ = run_k_fold(NFOLDS, seed)
    oof += oof_ / len(SEED)
    predictions += predictions_ / len(SEED)

#train[target_cols] = oof
#test[target_cols] = predictions



FOLD: 0, EPOCH: 0, train_loss: 0.4974690172011438
FOLD: 0, EPOCH: 0, valid_loss: 0.022945052332111766
FOLD: 0, EPOCH: 1, train_loss: 0.011115093108104624
FOLD: 0, EPOCH: 1, valid_loss: 0.008961276284285955
FOLD: 0, EPOCH: 2, train_loss: 0.00867213580109503
FOLD: 0, EPOCH: 2, valid_loss: 0.008761754765042236
FOLD: 0, EPOCH: 3, train_loss: 0.008496629830071892
FOLD: 0, EPOCH: 3, valid_loss: 0.008311535843781062
FOLD: 0, EPOCH: 4, train_loss: 0.008142763367462634
FOLD: 0, EPOCH: 4, valid_loss: 0.007963413731860262
FOLD: 0, EPOCH: 5, train_loss: 0.00803400184351789
FOLD: 0, EPOCH: 5, valid_loss: 0.00801062783492463
FOLD: 0, EPOCH: 6, train_loss: 0.008032015049020234
FOLD: 0, EPOCH: 6, valid_loss: 0.00808354735906635
FOLD: 0, EPOCH: 7, train_loss: 0.008073628630381132
FOLD: 0, EPOCH: 7, valid_loss: 0.00815630956952061
FOLD: 0, EPOCH: 8, train_loss: 0.008108449924359287
FOLD: 0, EPOCH: 8, valid_loss: 0.008013885228761605
FOLD: 0, EPOCH: 9, train_loss: 0.008132547848736462
FOLD: 0, EPOCH: 9, 

FOLD: 3, EPOCH: 4, train_loss: 0.008263469019067892
FOLD: 3, EPOCH: 4, valid_loss: 0.00805794345214963
FOLD: 3, EPOCH: 5, train_loss: 0.008097353583687673
FOLD: 3, EPOCH: 5, valid_loss: 0.008144160652799265
FOLD: 3, EPOCH: 6, train_loss: 0.008096351301756458
FOLD: 3, EPOCH: 6, valid_loss: 0.008106167827333723
FOLD: 3, EPOCH: 7, train_loss: 0.008116732081533342
FOLD: 3, EPOCH: 7, valid_loss: 0.008081641740032605
FOLD: 3, EPOCH: 8, train_loss: 0.008150190303938976
FOLD: 3, EPOCH: 8, valid_loss: 0.008065901350762163
FOLD: 3, EPOCH: 9, train_loss: 0.008173899124443962
FOLD: 3, EPOCH: 9, valid_loss: 0.00815456872805953
FOLD: 3, EPOCH: 10, train_loss: 0.008175398523419879
FOLD: 3, EPOCH: 10, valid_loss: 0.00807666927576065
FOLD: 3, EPOCH: 11, train_loss: 0.008188430948749832
FOLD: 3, EPOCH: 11, valid_loss: 0.008051856820072447
FOLD: 3, EPOCH: 12, train_loss: 0.00817532798923228
FOLD: 3, EPOCH: 12, valid_loss: 0.00813797085679003
FOLD: 3, EPOCH: 13, train_loss: 0.008161454714591737
FOLD: 3, E

FOLD: 1, EPOCH: 11, train_loss: 0.00815930357877759
FOLD: 1, EPOCH: 11, valid_loss: 0.008142231178603002
FOLD: 1, EPOCH: 12, train_loss: 0.008142835011813735
FOLD: 1, EPOCH: 12, valid_loss: 0.008063157820808036
FOLD: 1, EPOCH: 13, train_loss: 0.008151179226790217
FOLD: 1, EPOCH: 13, valid_loss: 0.008067976324153799
FOLD: 1, EPOCH: 14, train_loss: 0.008128372679019103
FOLD: 1, EPOCH: 14, valid_loss: 0.008040974701621703
FOLD: 1, EPOCH: 15, train_loss: 0.008102689086851002
FOLD: 1, EPOCH: 15, valid_loss: 0.007993438533906426
FOLD: 1, EPOCH: 16, train_loss: 0.008055003886313541
FOLD: 1, EPOCH: 16, valid_loss: 0.00793230871536902
FOLD: 1, EPOCH: 17, train_loss: 0.008020019168844041
FOLD: 1, EPOCH: 17, valid_loss: 0.007863059200878654
FOLD: 1, EPOCH: 18, train_loss: 0.007968797220214121
FOLD: 1, EPOCH: 18, valid_loss: 0.007883979220475469
FOLD: 1, EPOCH: 19, train_loss: 0.007928052043164338
FOLD: 1, EPOCH: 19, valid_loss: 0.007818925700017383
FOLD: 1, EPOCH: 20, train_loss: 0.00785241778804

FOLD: 4, EPOCH: 18, train_loss: 0.007980306447227147
FOLD: 4, EPOCH: 18, valid_loss: 0.007881399470248393
FOLD: 4, EPOCH: 19, train_loss: 0.0079203595245338
FOLD: 4, EPOCH: 19, valid_loss: 0.007855398859828711
FOLD: 4, EPOCH: 20, train_loss: 0.007847801431257656
FOLD: 4, EPOCH: 20, valid_loss: 0.007782917296780007
FOLD: 4, EPOCH: 21, train_loss: 0.007782115101598311
FOLD: 4, EPOCH: 21, valid_loss: 0.007769864185580185
FOLD: 4, EPOCH: 22, train_loss: 0.007711876527932675
FOLD: 4, EPOCH: 22, valid_loss: 0.007699383635606085
FOLD: 4, EPOCH: 23, train_loss: 0.007642601802270266
FOLD: 4, EPOCH: 23, valid_loss: 0.007671194603400571
FOLD: 4, EPOCH: 24, train_loss: 0.0076089349433617745
FOLD: 4, EPOCH: 24, valid_loss: 0.00766878972894379
FOLD: 0, EPOCH: 0, train_loss: 0.49730211357329634
FOLD: 0, EPOCH: 0, valid_loss: 0.01959537352834429
FOLD: 0, EPOCH: 1, train_loss: 0.010884349943017181
FOLD: 0, EPOCH: 1, valid_loss: 0.008963168039917946
FOLD: 0, EPOCH: 2, train_loss: 0.008672350022158977
FO

FOLD: 3, EPOCH: 2, train_loss: 0.008688414408622877
FOLD: 3, EPOCH: 2, valid_loss: 0.012900575755962304
FOLD: 3, EPOCH: 3, train_loss: 0.00880666017073436
FOLD: 3, EPOCH: 3, valid_loss: 0.008635957539081574
FOLD: 3, EPOCH: 4, train_loss: 0.00823647571741131
FOLD: 3, EPOCH: 4, valid_loss: 0.00808541722861784
FOLD: 3, EPOCH: 5, train_loss: 0.008141584653893242
FOLD: 3, EPOCH: 5, valid_loss: 0.008006088422345264
FOLD: 3, EPOCH: 6, train_loss: 0.008104111429006942
FOLD: 3, EPOCH: 6, valid_loss: 0.00805051526320832
FOLD: 3, EPOCH: 7, train_loss: 0.008110606121709165
FOLD: 3, EPOCH: 7, valid_loss: 0.008110170611845595
FOLD: 3, EPOCH: 8, train_loss: 0.008123621347940702
FOLD: 3, EPOCH: 8, valid_loss: 0.008138879335352353
FOLD: 3, EPOCH: 9, train_loss: 0.008147551225957231
FOLD: 3, EPOCH: 9, valid_loss: 0.008112463062363011
FOLD: 3, EPOCH: 10, train_loss: 0.008160714552724276
FOLD: 3, EPOCH: 10, valid_loss: 0.008109098459993089
FOLD: 3, EPOCH: 11, train_loss: 0.008159620933693604
FOLD: 3, EPOC

FOLD: 1, EPOCH: 9, train_loss: 0.00815889710371477
FOLD: 1, EPOCH: 9, valid_loss: 0.008246322441846133
FOLD: 1, EPOCH: 10, train_loss: 0.008180874463953618
FOLD: 1, EPOCH: 10, valid_loss: 0.008111512607761792
FOLD: 1, EPOCH: 11, train_loss: 0.008152379154943037
FOLD: 1, EPOCH: 11, valid_loss: 0.008252615933971745
FOLD: 1, EPOCH: 12, train_loss: 0.008181970272942082
FOLD: 1, EPOCH: 12, valid_loss: 0.008080237984125103
FOLD: 1, EPOCH: 13, train_loss: 0.008167377423387075
FOLD: 1, EPOCH: 13, valid_loss: 0.00807769527392728
FOLD: 1, EPOCH: 14, train_loss: 0.008150569460206274
FOLD: 1, EPOCH: 14, valid_loss: 0.008023641351610423
FOLD: 1, EPOCH: 15, train_loss: 0.008099187182370519
FOLD: 1, EPOCH: 15, valid_loss: 0.008043940644711256
FOLD: 1, EPOCH: 16, train_loss: 0.008078479619723732
FOLD: 1, EPOCH: 16, valid_loss: 0.007975432183593511
FOLD: 1, EPOCH: 17, train_loss: 0.008027267470226987
FOLD: 1, EPOCH: 17, valid_loss: 0.007938041990356786
FOLD: 1, EPOCH: 18, train_loss: 0.0079747824727193

FOLD: 4, EPOCH: 16, train_loss: 0.00807354474723663
FOLD: 4, EPOCH: 16, valid_loss: 0.00801127181787576
FOLD: 4, EPOCH: 17, train_loss: 0.00802840528321331
FOLD: 4, EPOCH: 17, valid_loss: 0.007948008153055396
FOLD: 4, EPOCH: 18, train_loss: 0.007972465237548602
FOLD: 4, EPOCH: 18, valid_loss: 0.007865660597703286
FOLD: 4, EPOCH: 19, train_loss: 0.00792703213021699
FOLD: 4, EPOCH: 19, valid_loss: 0.007813209375100477
FOLD: 4, EPOCH: 20, train_loss: 0.007865940844910085
FOLD: 4, EPOCH: 20, valid_loss: 0.007793273484068258
FOLD: 4, EPOCH: 21, train_loss: 0.0077852318232096195
FOLD: 4, EPOCH: 21, valid_loss: 0.007747380914432662
FOLD: 4, EPOCH: 22, train_loss: 0.007718189676845635
FOLD: 4, EPOCH: 22, valid_loss: 0.00771621617355517
FOLD: 4, EPOCH: 23, train_loss: 0.0076593330258206615
FOLD: 4, EPOCH: 23, valid_loss: 0.007672052178531885
FOLD: 4, EPOCH: 24, train_loss: 0.007635168525376829
FOLD: 4, EPOCH: 24, valid_loss: 0.007660989569766181


In [37]:
#normalize probabilities
for i in range(oof.shape[0]):
    oof[i,:] = oof[i,:]/np.sum(oof[i,:])
    
oof_original_target = np.matmul(oof, transition.to_numpy() )
prediction_original_target = np.matmul(predictions,transition.to_numpy())

In [38]:
#calculate train log loss
train_log_loss = 0
num_of_scored_targets = df['train_targets_scored'].shape[1]

trt_index = df_full_combine[df_full_combine.cp_type !='ctl_vehicle'].index
for i in range(num_of_scored_targets):
    y_true = df['train_targets_scored'].iloc[trt_index,i].values
    y_pred = oof_original_target[:,i]
    train_log_loss += log_loss(y_true, y_pred)/num_of_scored_targets
train_log_loss

0.01781585245107664

In [40]:
prediction_original_target.shape
scored_target_cols = df['sample_submission'].columns
sub = df['sample_submission'].copy()
sub[scored_target_cols] = prediction_original_target[:,:len(scored_target_cols)]


In [41]:
df['test_features'].reset_index(drop = True, inplace = True)
test_ctrl_index = df['test_features'][df['test_features'].cp_type == 'ctl_vehicle'].index

sub.iloc[test_ctrl_index,:] = 0

In [ ]:
def prob_clip(df, floor, ceiling):
    for c in range(df.shape[1]):
        df.iloc[:,c] = np.where(df.iloc[:,c]< bar, bar, df.iloc[:,c])
        df.iloc[:,c] = np.where(df.iloc[:,c] > ceiling, ceiling, df.iloc[:,c])
    return df

In [ ]:
sub_clipped = prob_clip(sub, 0.00015, 0.995)

In [42]:
sub_clipped.to_csv('submission.csv',index = True)